# Beautiful Soup

In [ ]:
from bs4 import BeautifulSoup
import requests

TechwithTime T3 Navigating the HTML tree
https://www.youtube.com/watch?v=lC6mucyD17k

In [ ]:
from bs4 import BeautifulSoup
import requests

url = 'https://coinmarketcap.com/'
result = requests.get(url).text
doc = BeautifulSoup(result, 'html.parser')

tbody = doc.tbody #in the website all price info is stored inside a huge table call tbody
trs = tbody.contents #grab content inside the table, returning a list

#Traversing different levels
trs[1].previous_sibling #return trs[0]
trs[1].next_sibling #return trs[2]
trs[1].next_siblings #return a generator object trs[1:]

trs[0].parent.name #tbody
list(trs[0].descendants)
list(trs[0].contents)
list(trs[0].children)

#Getting crytoprices
prices = {}

for tr in trs[:10]:
    name, price = tr.contents[2:4]
    fixed_name = name.p.string
    fixed_price = price.a.get_text(strip=True)
    prices[fixed_name] = fixed_price
    
print(prices)

TechwithTime T4 Finding the Best GPU price https://www.youtube.com/watch?v=zAEfWiC_KBU

In [ ]:
import re

search_term = input("What product do you want to search for: ")
url = f"https://www.newegg.com/p/pl?d={search_term}&N=4131"
page = requests.get(url).text
doc = BeautifulSoup(page, 'html.parser')

page_text = doc.find(class_='list-tool-pagination-text').strong
pages = int(str(page_text).split('/')[-2].split('>')[-1][:-1])

items_found = {}

for page in range(1, pages + 1):
    url = f"https://www.newegg.com/p/pl?d={search_term}&N=4131&page={page}"
    page = requests.get(url).text
    doc = BeautifulSoup(page, 'html.parser')
    
    div = doc.find(class_='item-cells-wrap border-cells items-grid-view four-cells expulsion-one-cell')
    items = div.find_all(text=re.compile(search_term))
    for item in items:
        parent = item.parent
        if parent.name != "a":
            continue
            
        link = parent['href']
        next_parent = item.find_parent(class_="item-container")
        try:
            price = next_parent.find(class_="price-current").find('strong').string
            items_found[item] = {'price': int(price.replace(',','')), 'link': link}
        except:
            pass
        

sorted_items = sorted(items_found.items(), key=lambda x: x[1]['price'])

# for item in sorted_items:
#     print(item[0])
#     print(f"${item[1]['price']}")

# Selenium

In [ ]:
pip install webdriver-manager

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import time

options = Options()
options.add_experimental_option("detach", True)

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

driver.get("https://www.neuralnine.com/")
driver.maximize_window()
print(driver.title)

links = driver.find_elements("xpath", "//a[@href]")
for link in links:
    if "Books" in link.get_attribute("innerHTML"):
        link.click()
        break

book_links = driver.find_elements("xpath", "//div[contains(@class, 'elementor-column-wrap')][.//h2[text()[contains(.,'7 IN 1')]]][count(.//a)=2]//a")

book_links[0].click()
driver.switch_to.window(driver.window_handles[1]) #switch to 2nd tab

time.sleep(5)
buttons = driver.find_elements("xpath", "//a[.//span[text()[contains(.,'Tascjenbuch')]]]//span[text()[contains(.,$)]]")
for button in buttons:
    print(button.get_attribute("innerHTML"))
driver.quit()

search = driver.find_element("name", "s")
search.send_keys("test")
search.send_keys(Keys.RETURN)

print(driver.page_source)


### Important technique using Selenium -> driver.execute(js code) to do dynamic scraping like scrolling down

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import time
from bs4 import BeautifulSoup
import requests
import pandas as pd

options = Options()
options.add_experimental_option("detach", True)

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

driver.get("https://anilist.co/search/manga?format=MANGA&publishing%20status=FINISHED&country%20of%20origin=KR&sort=SCORE_DESC&year=2022")

last_height = driver.execute_script("return document.body.scrollHeight")
print(last_height)

while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    print("ran", end='')
    time.sleep(10)
    new_height = driver.execute_script("return document.body.scrollHeight")
    print(new_height)
    if new_height == last_height:
        break
    else:
        last_height = new_height

doc = BeautifulSoup(driver.page_source, 'html.parser')

## Important Selenium Fix for Chrome version 115+

Since chrome version 114+, WebDriverManager has migrated it's install place and stuff has messed up so you can't do the normal stuff normally, here is some fix I found online.

First fix is to add service = Service() line

In [ ]:
from selenium import webdriver

from selenium.webdriver.chrome.service import Service

service = Service() # this is the important line of code!

options = webdriver.ChromeOptions()
options.add_argument("--headless=new")

driver = webdriver.Chrome(service=service, options=options)

driver.get('https://news.google.com/topics/CAAqJggKIiBDQkFTRWdvSUwyMHZNRGx1YlY4U0FtVnVHZ0pWVXlnQVAB?hl=en-US&gl=US&ceid=US%3Aen')

2nd fix - specify a lower version of chrome in selenium

In [ ]:
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager(version="114.0.5735.90").install()),
                         options=option)

# XPath

Test your xpath query: http://xpather.com/

**Absolute referencing:** '/' eg, body/div (basically only search all div under 1 indent of body <br>
**Relative referencing:** '//' eg, body//div (return all the div tags under body)


**Self-reference:** '.' eg, //a/./@title (basically grab the 'title' attribute inside that a tag)<br>
**Parent selection:** '..' eg, //a/../@class (get the 'class' of the parent of the a tag)

**Grabbing text:** 'text()' eg, //a/text() (get the text inside the a tag)<br>
**Grabbing attributes:** '/@atr' eg, //a/@href (get the href inside the a tag)<br>


**Filtering:** <br>

In [ ]:
//a[@title]/@href "get the links of all <a> tag that contains a title"

a[*] "return <a> tag that have something in them that isn't just text, eg. img"

//*[@*] "select everything that contains something as attribute"

//div[@class='col-sm-8 h1'] 'select all div that has the exact class of "col-sm-8 h1'

//div[contains(@class, 'col-sm-8')] 'select all div that contains the class "col-sm-8"'

//*[text()[contains(., 'Book')]] 'find any element that contains the text "book"'

//*[count(.//li)=20] 'return the item that contains 20 list items in its children'

//div/following-sibling::div 'return div block that is after another div block in its scope'
//div/preceding-sibling::div 'return div block that is before another div block in its scope'
        

# Automation with python - freecodecamp

Youtube: https://www.youtube.com/watch?v=PXMJ6FS7llk

## Extracting table from websites using Pandas

In [ ]:
import pandas as pd
# use read_html to return all tables from the website
simpsons = pd.read_html('https://en.wikipedia.org/wiki/List_of_The_Simpsons_episodes')

## Extracting table from PDF 

In [ ]:
pip install tk
pip install ghostscript
pip install camelot-py

In [ ]:
import camelot

tables = camelot.read_pdf('foo.pdf', pages='1')
tables[0].export('foo.csv', f='csv', compress=True)

## Seleniem project - Daily auto execution

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
import pandas as pd
from datetime import datetime
import os
import sys

# Preparing script before we convert it to executable
application_path = os.path.dirname(sys.executable)

# get date in format MMDDYYYY
now = datetime.now()
month_day_year = now.strftime("%m%d%Y")

web = 'https://www.thesun.co.uk/sport/football/'
path = '/Users/frankandrade/Downloads/chromedriver'  # introduce path here

# Headless mode
options = Options()
options.headless = True
driver_service = Service(executable_path=path)
driver = webdriver.Chrome(service=driver_service, options=options)
driver.get(web)

containers = driver.find_elements(by='xpath', value='//div[@class="teaser__copy-container"]')

titles = []
subtitles = []
links = []
for container in containers:
    title = container.find_element(by='xpath', value='./a/h2').text
    subtitle = container.find_element(by='xpath', value='./a/p').text
    link = container.find_element(by='xpath', value='./a').get_attribute('href')
    titles.append(title)
    subtitles.append(subtitle)
    links.append(link)

# Exporting data to the same folder where the executable will be located
my_dict = {'title': titles, 'subtitle': subtitles, 'link': links}
df_headlines = pd.DataFrame(my_dict)
file_name = f'football_headlines_{month_day_year}.csv'
final_path = os.path.join(application_path, file_name)
df_headlines.to_csv(final_path)

driver.quit()

To convert the script to be able to auto execute daily, you need to run it as python file and convert it to exe. Follow the following steps (make sure your current directory is in the project directory [able to see the file])

1. `pip install pyinstaller`
2. `pyinstaller --onefile *name of py file*`

After doing the two steps you should see build and dict folder in your directory, the exe is the one you see inside dict folder. If you want to run it, right click it and open with terminal.

Now we want it to run it automatically daily

1. `crontab -e`
2. You can test the cron schedule at https://crontab.guru/
3. Go back to the terminal, for example, if you want it to run in 9am everyday, paste 09\*\*\* in the vim editor and press **i** and a space  -> drag the exe file into the terminal to get the path of the exe file (a space between 09\*\*\* and the path of exe). Then press **esc** and **:wq**
4. `contab -l` to see if the command is created

### Important! Crontab is not recommended for mac due to the security system of macos, use launchd instead

1. Create .plist file, create it by typing `touch com.scraping.googlenews.plist` in the terminal in this example 

<?xml version="1.0" encoding="UTF-8"?>
<!DOCTYPE plist PUBLIC "-//Apple//DTD PLIST 1.0//EN" "http://www.apple.com/DTDs/PropertyList-1.0.dtd">
<plist version="1.0">
  <dict>
    <key>Label</key>
    <string>com.scraping.googlenews</string>
    <key>ProgramArguments</key>
    <array>
      <string>/Users/jadonng/opt/anaconda3/bin/python</string>
      <string>/Users/jadonng/Desktop/Computer_Science/Data_Scraping/Google_News/GNScraping_nocontent.py</string>
    </array>
    <key>RunAtLoad</key>
    <true/>
    <key>StandardOutPath</key>
    <string>/Users/jadonng/Desktop/Computer_Science/Data_Scraping/Google_News/stdout.log</string>
    <key>StandardErrorPath</key>
    <string>/Users/jadonng/Desktop/Computer_Science/Data_Scraping/Google_News/stderror.log</string>
    <key>StartCalendarInterval</key>
    <dict>
      <key>Hour</key>
      <integer>13</integer>
      <key>Minute</key>
      <integer>16</integer>
    </dict>
  </dict>
</plist>

2. Move the file into Library/LaunchAgent

3. In terminal, load the plist file by typing `launchctl load plistfilepath` (drag the plist file in)

To unload it type `launchctl unload filepath`

To check all plist running in the pc, type `launchctl list`

### Since the first two method really doesn't work, there is a third method

https://www.youtube.com/watch?v=nVlOapHc-kg